# Drizzled PSFs from _HST_/ACS Imaging

This example notebook shows how `spike` interacts with _HST_/ACS imaging to generate a drizzled PSF. This notebook uses observations from [this program](https://archive.stsci.edu/cgi-bin/mastpreview?mission=hst&amp;dataid=J8PU42010). The easiest way to find these data in the MAST archive is by searching for observation ID j8pu42010. `spike` uses calibrated, but not-yet co-added images (i.e., "Level 2" data products), so be sure that your download includes 'flc' files. If you download all of the files associated with this program, your working directory will include:
- j8pu42ecq_flc.fits
- j8pu42egq_flc.fits
- j8pu42esq_flc.fits
- j8pu42evq_flc.fits

The principle of this example notebook is the same regardless of dataset, though, so it can be used as a direct guide for use with other data, too.

_NOTE: Environment variables are not always read in properly within Jupyter notebooks/JupyterLab -- see [here](https://github.com/avapolzin/spike/blob/master/example_notebooks/README.md) for some additional details of how to access them._

In [1]:
from spike.psf import hst # import the relevant top-level module from spike

datapath = '/path/to/acs/data'
outputpath = 'psfs' #default is /psfs, defined from your working directory

Note that if you choose to create drizzled PSFs for the same data and object location using multiple PSF generation methods, you will want to define a new 'savedir' each time to avoid conflicts between files with the same name.

We'll also define an object -- in this case, a set of coordinates -- for which the PSF will be generated.

In [2]:
obj = '10:00:33.0178 +02:09:52.304' #can also be a resolvable name or coordinates in decimal degress

Now, we can call `spike.psf.hst` -- as an initial example we'll use the most minimal inputs. By default, in addition to saving drizzled PSFs and any intermediate products in the directory given by 'savedir' (here the argument will be the outputpath variable we definied), this function will return a dictionary that stores model PSFs indexed by filter and object. 

If `returnpsf = 'full'` (default), the arrays in the dictionary will be the full drizzled image field of view. If `returnpsf = 'crop'`, the arrays in the dictionary will be the cutout region around the object location with the size of the cutout determined by the 'cutout_fov' argument; the cropped region around the drizzled PSF will also be saved as a FITS file, including its WCS information (can be toggled off with `savecutout = False`).

`spike` assumes that the input files have not yet been "tweaked" and handles that step for you. If your input images have been tweaked _or_ if you would like to skip this step for other reasons, specify `pretweaked = True` in calling `spike.psf.hst`.

The outputs are very long due to all of the print statements from the tweak, PSF generation, and drizzling steps. Outputs have been cleared for cells where `spike.psf.hst` was run.

In [ ]:
dpsf = hst(img_dir = datapath, obj = obj, img_type = 'flc', inst = 'ACS', camera = 'WFC', savedir = outputpath, pretweaked = True)

In this case dpsf will be very simple, since we are only looking at one set of coordinates and one filter:

In [4]:
dpsf

{'10:00:33.0178 +02:09:52.304': {'F475W': array([[ 0.,  0.,  0., ..., nan, nan, nan],
         [ 0.,  0.,  0., ..., nan, nan, nan],
         [ 0.,  0.,  0., ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ...,  0.,  0.,  0.],
         [nan, nan, nan, ...,  0.,  0.,  0.],
         [nan, nan, nan, ...,  0.,  0.,  0.]],
        shape=(2359, 4162), dtype='>f4')}}

By default, PSFs are generated for `spike.psf.hst` by `TinyTim` using a blackbody model of a G5V star at 6000 K, but users have complete control over the specifics of the PSF generation. To change the parameters within a method (e.g., updating model arguments, setting different detection thresholds for the empirical PSFs, ...), `spike.psf.hst` directly takes keyword arguments for specified 'method'. Details of allowed arguments are available in the `spike.psfgen` [documentation](https://spike-psf.readthedocs.io/en/latest/psfgen.html). As an example, we will change the `TinyTim` model inputs to use an O6 star at 45000 K:

In [ ]:
dpsf = hst(img_dir = datapath, obj = obj, img_type = 'flc', inst = 'ACS', camera = 'WFC', savedir = outputpath, pretweaked = True, 
          listchoice = 'O6', temp = 45000)

`spike` can also be used to generate PSFs for multiple objects/coordinates simultaneously.

In [ ]:
obj = ['10:00:33.0178 +02:09:52.304', '10:00:28.1798 +02:08:40.507']

dpsf = hst(img_dir = datapath, obj = obj, img_type = 'flc', inst = 'ACS', camera = 'WFC', savedir = outputpath, pretweaked = True)

In [7]:
dpsf

{'10:00:33.0178 +02:09:52.304': {'F475W': array([[ 0.,  0.,  0., ..., nan, nan, nan],
         [ 0.,  0.,  0., ..., nan, nan, nan],
         [ 0.,  0.,  0., ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ...,  0.,  0.,  0.],
         [nan, nan, nan, ...,  0.,  0.,  0.],
         [nan, nan, nan, ...,  0.,  0.,  0.]],
        shape=(2359, 4162), dtype='>f4')},
 '10:00:28.1798 +02:08:40.507': {'F475W': array([[ 0.,  0.,  0., ..., nan, nan, nan],
         [ 0.,  0.,  0., ..., nan, nan, nan],
         [ 0.,  0.,  0., ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ...,  0.,  0.,  0.],
         [nan, nan, nan, ...,  0.,  0.,  0.],
         [nan, nan, nan, ...,  0.,  0.,  0.]],
        shape=(2359, 4162), dtype='>f4')}}

One advantage of `spike` is that it can be used flexibly with a variety of different PSF generation methods -- to change which one is used, simply use the 'method' argument. Built-in options are:
- 'TinyTim’
- ‘TinyTim_Gillis’
- ‘STDPSF’
- ‘epsf’
- ‘PSFEx’

If `method = 'USER'`, then the keyword argument 'usermethod' must also be specified, pointing to either a function or a directory that houses model PSFs that follow the \[imgprefix]\_\[coords]\_\[band]\_topsf.fits, e.g., imgprefix_23.31+30.12_F814W_topsf.fits or imgprefix_195.78-46.52_F555W_topsf.fits naming scheme.

We'll use STDPSFs below and have `spike` output cutouts around the drizzled PSF.

In [ ]:
obj = '10:00:33.0178 +02:09:52.304'

dpsf = hst(img_dir = datapath, obj = obj, img_type = 'flc', inst = 'ACS', camera = 'WFC', 
           savedir = outputpath, pretweaked = True, method = 'STDPSF', returnpsf = 'crop')

In [10]:
dpsf

{'10:00:33.0178 +02:09:52.304': {'F475W': array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], shape=(151, 151), dtype='>f4')}}

`spike` can handle most plotting automatically and generates PSFs regardless of whether they are plotted. In most cases, `spike` is the only package you'll actually need to both create and visualize your PSFs.